In [1]:
import pandas as pd

In [2]:

#get data file
!wget -q -O data.csv "https://docs.google.com/uc?export=download&id=1mFCRDIWsokNF02WBk77m0CIE7ZfRBp_k"

In [3]:
data = pd.read_csv('data.csv')
data.head()

,text_id_kaggle,full_text,gender,grade,race_ethnicity,num_words,num_words2,num_words3,num_sent,num_para,...,task,SES,prompt,Overall,Cohesion,Syntax,Vocabulary,Phraseology,Grammar,Conventions
0,5AA45740A848,"Dear, TEACHER_NAME\n\nI think phone policy at ...",Male,8,Hispanic/Latino,111,122,118,7,2,...,Independent,Economically disadvantaged,Cell phones at school,3.0,3.5,2.5,3.5,3.0,3.5,2.5
1,790891A35047,"Dear, Principal\r\n\r\nIn my opinion, I think ...",Female,8,Hispanic/Latino,99,105,102,6,2,...,Independent,Economically disadvantaged,Cell phones at school,3.0,2.5,3.0,4.0,3.5,3.0,3.5
2,C254D74362A3,PHONES\n\nDear principal students should have ...,Female,8,Hispanic/Latino,121,134,128,4,2,...,Independent,Economically disadvantaged,Cell phones at school,3.0,2.5,3.0,3.0,3.0,3.5,3.0
3,0985602832CD,phones\n\ni think phones should be allowed in ...,Male,8,Hispanic/Latino,182,202,192,2,2,...,Independent,Economically disadvantaged,Cell phones at school,3.0,2.0,2.5,3.0,3.0,3.0,2.5
4,3AB3FC197550,Do you really think students need cell phones ...,Female,8,Hispanic/Latino,192,210,201,11,4,...,Independent,Not economically disadvantaged,Cell phones at school,3.0,2.0,3.0,3.0,3.0,3.0,2.0


In [4]:
overall = data.rename(columns = {'text_id_kaggle':'student_id', 'Overall':'score' })[['student_id', 'full_text', 'prompt','score']]
cohesion = data.rename(columns ={'text_id_kaggle':'student_id', 'Cohesion':'score'})[['student_id', 'full_text', 'prompt','score']]
syntax = data.rename(columns = {'text_id_kaggle':'student_id', 'Syntax':'score'})[['student_id', 'full_text', 'prompt','score']]
vocab = data.rename(columns = {'text_id_kaggle':'student_id', 'Vocabulary':'score'})[['student_id', 'full_text', 'prompt','score']]
phraseology = data.rename(columns = {'text_id_kaggle':'student_id', 'Phraseology':'score'})[['student_id', 'full_text', 'prompt','score']]
grammar = data.rename(columns = { 'text_id_kaggle':'student_id', 'Grammar':'score'})[['student_id', 'full_text', 'prompt','score']]
conventions = data.rename(columns = {'text_id_kaggle':'student_id', 'Conventions':'score'})[['student_id', 'full_text', 'prompt','score']]

In [5]:
overall.head()

,student_id,full_text,prompt,score
0,5AA45740A848,"Dear, TEACHER_NAME\n\nI think phone policy at ...",Cell phones at school,3.0
1,790891A35047,"Dear, Principal\r\n\r\nIn my opinion, I think ...",Cell phones at school,3.0
2,C254D74362A3,PHONES\n\nDear principal students should have ...,Cell phones at school,3.0
3,0985602832CD,phones\n\ni think phones should be allowed in ...,Cell phones at school,3.0
4,3AB3FC197550,Do you really think students need cell phones ...,Cell phones at school,3.0


In [27]:
diff_arrays = []
for section in [overall, cohesion, syntax, vocab, phraseology, grammar, conventions]:
    diff = pd.DataFrame(columns=['id_1', 'id_2', 'text_1', 'text_2', '1_better', 'prompt'])

    prompts = section['prompt'].unique()

    for prompt in prompts:
        prompt_essays = section[section['prompt'] == prompt]

        if len(prompt_essays) >= 2:
            original = prompt_essays.sample(n=min(100, len(prompt_essays)//2), random_state=420)
            remaining = prompt_essays[~prompt_essays.index.isin(original.index)]

            other = remaining.sample(n=min(len(original), len(remaining)), random_state=69)

            for i in range(min(len(original), len(other))):
                id_1 = original.iloc[i]['student_id']
                text_1 = original.iloc[i]['full_text']
                id_2 = other.iloc[i]['student_id']
                text_2 = other.iloc[i]['full_text']

                is_1_better = int(original.iloc[i]['score'] > other.iloc[i]['score'])

                new_row = pd.DataFrame({
                    'id_1': [id_1],
                    'id_2': [id_2],
                    'text_1': [text_1],
                    'text_2': [text_2],
                    '1_better': [is_1_better],
                    'prompt': [prompt]
                })

                diff = pd.concat([diff, new_row], ignore_index=True)

    diff_arrays.append(diff)

In [28]:
diff_arrays[0]

,id_1,id_2,text_1,text_2,1_better,prompt
0,EBAC7604E958,ACDF0CAAF77C,"Dear Principal,\n\nPrincipal I will suggest po...",Dear TEACHER_NAME\n\nI've realized you have be...,0,Cell phones at school
1,4FE8CD4987B6,F7B655834981,I think policy one is good because at lunch st...,she is not to bring a phones in school because...,1,Cell phones at school
2,4210C4CA9806,CEC71375C061,"Dear Principal,\n\nI think you should let us b...",Policy1: I think we should be allowed to use o...,1,Cell phones at school
3,223FCA37F79D,1B2B4A98C5E6,We should allow the students to bring there ce...,Policy 2: Do not allow students to have phones...,1,Cell phones at school
4,1D51BCF69AD3,3AB3FC197550,"dear principal,\n\ni think all students should...",Do you really think students need cell phones ...,0,Cell phones at school
...,...,...,...,...,...,...
2579,8A68943E1657,14C4130F90C2,People wanted interfere in young people's live...,Do you think that teenager need time to be in ...,0,Curfews for teenagers
2580,947DF52352AB,2B6BA642E4F5,Adoption of a 10 P.M. weekday curfew and midni...,"Firstly, Adopting is good for teenagers and ma...",1,Curfews for teenagers
2581,0D382343F3D7,C79292B5F78A,Why do curfews interfere with young people's l...,My thoughts on this are that it is good time a...,1,Curfews for teenagers
2582,3F3DC123EDF6,CBFD5CD8BCD9,"Do curfews keep teenagers out of trouble, or d...","Yes, I believe curfew can keep citizens out of...",1,Curfews for teenagers


In [29]:
section_names = ['overall', 'cohesion', 'syntax', 'vocab', 'phraseology', 'grammar', 'conventions']

for i, diff_df in enumerate(diff_arrays):
    section_name = section_names[i]
    file_path = f'{section_name}_paired_essays.csv'
    diff_df.to_csv(file_path, index=False)
    print(f"Saved {len(diff_df)} paired essays to {file_path}")

Saved 2584 paired essays to overall_paired_essays.csv
Saved 2584 paired essays to cohesion_paired_essays.csv
Saved 2584 paired essays to syntax_paired_essays.csv
Saved 2584 paired essays to vocab_paired_essays.csv
Saved 2584 paired essays to phraseology_paired_essays.csv
Saved 2584 paired essays to grammar_paired_essays.csv
Saved 2584 paired essays to conventions_paired_essays.csv
